---

## Implementation of Livy API

In [ ]:
import sys

In [ ]:
sys.path.insert(0, '../')

In [ ]:
from livy_submit import livy_api

---
## Manual QA of Livy API

Configure the API object

In [ ]:
server_url = 'ip-172-31-20-241.ec2.internal'

In [ ]:
api = livy_api.LivyAPI(server_url=server_url, port=8998)

Upload files to HDFS

In [ ]:
import os
from hdfs.ext.kerberos import KerberosClient

In [ ]:
namenode_connection_url = f'http://{server_url}:50070'
run_file = os.path.abspath('pi.py')
hdfs_dir = '/user/edill/pi.py'
job_name = 'edill-pi'
spark_config = {}

In [ ]:
run_file

In [ ]:
# 1. upload run_file to hdfs_dir
client = KerberosClient(f'{namenode_connection_url}')

Make sure that `pi.py` exists in the home directory of your user

In [ ]:
client.list('/user/edill')

In [ ]:
file_on_hdfs = client.upload(hdfs_path='', local_path=run_file, overwrite=True)
file_on_hdfs

Start using the Livy Python API

See what info we have available. This will likely be an output that looks like this:
```
(0, 0, {})
```

In [ ]:
api.all_info()

This one should throw a stack trace with:
```
HTTPError: 404 Client Error: Not Found for url: http://ip-172-31-20-241.ec2.internal:8998/batches/26
```

In [ ]:
api.info(26)

---
## Submit

Submit your livy job

In [ ]:
job1 = api.submit(file=file_on_hdfs, name=job_name)
job1

Check on its info

In [ ]:
api.info(job1.id)

Submit another job so we can have additional info in the all_info function

In [ ]:
job2 = api.submit(file=file_on_hdfs, name=job_name)
job2

Check all available batch jobs. You should have two, if you've run the cells in order and only once each.

In [ ]:
api.all_info()

---
Look at the logs for the first job. Make sure you see this log line:
```
  'Pi is roughly 3.139560',
```

In [ ]:
api.logs(job1.id)

---
Look at the logs for the second job. Make sure you see this log line:
```
  'Pi is roughly 3.139560',
```

In [ ]:
api.logs(job2.id)

---
Now, let's start a third job so we can kill it

In [ ]:
job3 = api.submit(file=file_on_hdfs, name=job_name)
job3

And let's kill it

In [ ]:
api.kill(job3.id)

If you see this above

```
{'msg': 'deleted'}
```

Then things are working as expected